<a href="https://colab.research.google.com/github/Dkaka/Deeplab_colab_demo/blob/main/W1_SImplified_Kitti_Distorted(RAW)_of_Local_Zip_of_DeepLab_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview

This colab demonstrates the steps to use the DeepLab model to perform semantic segmentation on a sample input image. Expected outputs are semantic labels overlayed on the sample image.

### About DeepLab
The models used in this colab perform semantic segmentation. Semantic segmentation models focus on assigning semantic labels, such as sky, person, or car, to multiple objects and stuff in a single image.

In [1]:
dataset="05"

## Import Libraries

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Jul  2 22:39:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import os
from io import BytesIO
import tarfile
import tempfile
from six.moves import urllib

from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
%tensorflow_version 1.x
# %tensorflow_version 1.x
import tensorflow as tf
# config = tf.compat.v1.ConfigProto()
# config.gpu_options.allow_growth = True
# sess = tf.compat.v1.Session(config=config)
# sess.as_default()

print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


## Import helper methods
These methods help us perform the following tasks:
* Load the latest version of the pretrained DeepLab model
* Load the colormap from the PASCAL VOC dataset
* Adds colors to various labels, such as "pink" for people, "green" for bicycle and more
* Visualize an image, and add an overlay of colors on various regions

In [4]:
class DeepLabModel(object):
  """Class to load deeplab model and run inference."""

  INPUT_TENSOR_NAME = 'ImageTensor:0'
  OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'
  # INPUT_SIZE = 513
  # INPUT_SIZE = 1025
  # INPUT_SIZE = 1242
  INPUT_SIZE = 1226
  FROZEN_GRAPH_NAME = 'frozen_inference_graph'

  def __init__(self, tarball_path):
    """Creates and loads pretrained deeplab model."""
    self.graph = tf.Graph()

    graph_def = None
    # Extract frozen graph from tar archive.
    tar_file = tarfile.open(tarball_path)
    for tar_info in tar_file.getmembers():
      if self.FROZEN_GRAPH_NAME in os.path.basename(tar_info.name):
        file_handle = tar_file.extractfile(tar_info)
        graph_def = tf.GraphDef.FromString(file_handle.read())
        break

    tar_file.close()

    if graph_def is None:
      raise RuntimeError('Cannot find inference graph in tar archive.')

    with self.graph.as_default():
      tf.import_graph_def(graph_def, name='')

    self.sess = tf.Session(graph=self.graph)

  def run(self, image):
    """Runs inference on a single image.

    Args:
      image: A PIL.Image object, raw input image.

    Returns:
      resized_image: RGB image resized from original input image.
      seg_map: Segmentation map of `resized_image`.
    """
    width, height = image.size
    print (image.size)
    resize_ratio = 1.0 * self.INPUT_SIZE / max(width, height)
    print (resize_ratio)
    target_size = (int(resize_ratio * width), int(resize_ratio * height))
    resized_image = image.convert('RGB').resize(target_size, Image.ANTIALIAS)
    batch_seg_map = self.sess.run(
        self.OUTPUT_TENSOR_NAME,
        feed_dict={self.INPUT_TENSOR_NAME: [np.asarray(resized_image)]})
    seg_map = batch_seg_map[0]
    return resized_image, seg_map


def create_pascal_label_colormap():
  """Creates a label colormap used in PASCAL VOC segmentation benchmark.

  Returns:
    A Colormap for visualizing segmentation results.
  """
  colormap = np.zeros((256, 3), dtype=int)
  ind = np.arange(256, dtype=int)

  for shift in reversed(range(8)):
    for channel in range(3):
      colormap[:, channel] |= ((ind >> channel) & 1) << shift
    ind >>= 3

  return colormap


def label_to_color_image(label):
  """Adds color defined by the dataset colormap to the label.

  Args:
    label: A 2D array with integer type, storing the segmentation label.

  Returns:
    result: A 2D array with floating type. The element of the array
      is the color indexed by the corresponding element in the input label
      to the PASCAL color map.

  Raises:
    ValueError: If label is not of rank 2 or its value is larger than color
      map maximum entry.
  """
  if label.ndim != 2:
    raise ValueError('Expect 2-D input label')

  colormap = create_pascal_label_colormap()

  if np.max(label) >= len(colormap):
    raise ValueError('label value too large.')

  return colormap[label]


def vis_segmentation(image, seg_map):
  """Visualizes input image, segmentation map and overlay view."""
  plt.figure(figsize=(15, 5))
  grid_spec = gridspec.GridSpec(1, 4, width_ratios=[6, 6, 6, 1])

  plt.subplot(grid_spec[0])
  plt.imshow(image)
  plt.axis('off')
  plt.title('input image')

  plt.subplot(grid_spec[1])
  seg_image = label_to_color_image(seg_map).astype(np.uint8)
  plt.imshow(seg_image)
  plt.axis('off')
  plt.title('segmentation map')

  plt.subplot(grid_spec[2])
  plt.imshow(image)
  plt.imshow(seg_image, alpha=0.7)
  plt.axis('off')
  plt.title('segmentation overlay')

  unique_labels = np.unique(seg_map)
  ax = plt.subplot(grid_spec[3])
  plt.imshow(
      FULL_COLOR_MAP[unique_labels].astype(np.uint8), interpolation='nearest')
  ax.yaxis.tick_right()
  plt.yticks(range(len(unique_labels)), LABEL_NAMES[unique_labels])
  plt.xticks([], [])
  ax.tick_params(width=0.0)
  plt.grid('off')
  plt.show()


# LABEL_NAMES = np.asarray([
#     'background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus',
#     'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
#     'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tv'
# ])

# LABEL_NAMES = np.asarray([
#     'unlabeled', 'ego vehicle', 'out of roi', 'static', 'dynamic', 'ground', 'road',
#     'sidewalk', 'parking', 'rail track', 'building', 'wall', 'fence', 'guard rail', 'bridge',
#     'tunnel', 'pole', 'polegroup', 'traffic light', 'traffic sign', 'vegetation', 'terrain',
#      'sky', 'person', 'rider', 'car', 'truck', 'bus', 'caravan', 'trailer', 'train', 
#      'motorcycle', 'bicycle', 'license plate'
# ])

LABEL_NAMES = np.asarray([
    'road', 'sidewalk', 'building', 'wall', 'fence', 'pole', 'traffic light',
    'traffic sign', 'vegetation',  'terrain',
     'sky', 'person', 'rider', 'car', 'truck', 'bus', 'train', 
     'motorcycle', 'bicycle'
])

FULL_LABEL_MAP = np.arange(len(LABEL_NAMES)).reshape(len(LABEL_NAMES), 1)
FULL_COLOR_MAP = label_to_color_image(FULL_LABEL_MAP)

## Select a pretrained model
We have trained the DeepLab model using various backbone networks. Select one from the MODEL_NAME list.

In [5]:
MODEL_NAME = 'deeplabv3_cityscapes_train'  # @param ['xception65_ade20k_train','mobilenetv2_coco_voctrainaug', 'mobilenetv2_coco_voctrainval', 'xception_coco_voctrainaug', 'xception_coco_voctrainval','deeplabv3_cityscapes_train']

_DOWNLOAD_URL_PREFIX = 'http://download.tensorflow.org/models/'
_MODEL_URLS = {
    'mobilenetv2_coco_voctrainaug':
        'deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz',
    'mobilenetv2_coco_voctrainval':
        'deeplabv3_mnv2_pascal_trainval_2018_01_29.tar.gz',
    'xception_coco_voctrainaug':
        'deeplabv3_pascal_train_aug_2018_01_04.tar.gz',
    'xception_coco_voctrainval':
        'deeplabv3_pascal_trainval_2018_01_04.tar.gz',
        'deeplabv3_cityscapes_train':
        'deeplabv3_cityscapes_train_2018_02_06.tar.gz',
         'xception65_ade20k_train':
        'deeplabv3_xception_ade20k_train_2018_05_29.tar.gz'
}
_TARBALL_NAME = 'deeplab_model.tar.gz'
model_dir = tempfile.mkdtemp()
tf.gfile.MakeDirs(model_dir)

download_path = os.path.join(model_dir, _TARBALL_NAME)
print('downloading model, this might take a while...')
urllib.request.urlretrieve(_DOWNLOAD_URL_PREFIX + _MODEL_URLS[MODEL_NAME],
                   download_path)
print('download completed! loading DeepLab model...')

MODEL = DeepLabModel(download_path)
print('model loaded successfully!')

downloading model, this might take a while...
download completed! loading DeepLab model...
model loaded successfully!


#Unzip dataset from google drive

In [6]:
import zipfile
from google.colab import drive
drive.mount('/content/drive/')

!cp  /content/drive/My\ Drive/kitti/"$dataset".zip /tmp

# !tar -xvf  '/tmp/dataset-outdoors2_1024_16.tar' -C '/tmp/tum_vi/'
# !tar -xvf  '/content/drive/My Drive/tum_vi/dataset-outdoors2_1024_16.tar' -C '/tmp'





Mounted at /content/drive/


In [7]:
!unzip '/tmp/{dataset}.zip' -d '/tmp/'

Streaming output truncated to the last 5000 lines.
  inflating: /tmp/05/image_0/002572.png  
  inflating: /tmp/05/image_0/000456.png  
  inflating: /tmp/05/image_0/001165.png  
  inflating: /tmp/05/image_0/001387.png  
  inflating: /tmp/05/image_0/000481.png  
  inflating: /tmp/05/image_0/000716.png  
  inflating: /tmp/05/image_0/001593.png  
  inflating: /tmp/05/image_0/001861.png  
  inflating: /tmp/05/image_0/000381.png  
  inflating: /tmp/05/image_0/000000.png  
  inflating: /tmp/05/image_0/001144.png  
  inflating: /tmp/05/image_0/000721.png  
  inflating: /tmp/05/image_0/002665.png  
  inflating: /tmp/05/image_0/000728.png  
  inflating: /tmp/05/image_0/000452.png  
  inflating: /tmp/05/image_0/001998.png  
  inflating: /tmp/05/image_0/001004.png  
  inflating: /tmp/05/image_0/000012.png  
  inflating: /tmp/05/image_0/000147.png  
  inflating: /tmp/05/image_0/001804.png  
  inflating: /tmp/05/image_0/000651.png  
  inflating: /tmp/05/image_0/000404.png  
  inflating: /tmp/05/imag

## What's next

* Learn about [Cloud TPUs](https://cloud.google.com/tpu/docs) that Google designed and optimized specifically to speed up and scale up ML workloads for training and inference and to enable ML engineers and researchers to iterate more quickly.
* Explore the range of [Cloud TPU tutorials and Colabs](https://cloud.google.com/tpu/docs/tutorials) to find other examples that can be used when implementing your ML project.
* For more information on running the DeepLab model on Cloud TPUs, see the [DeepLab tutorial](https://cloud.google.com/tpu/docs/tutorials/deeplab).


In [8]:
  def convert(img, target_type_min, target_type_max, target_type):
    imin = img.min()
    imax = img.max()

    a = (target_type_max - target_type_min) / (imax - imin)
    b = target_type_max - a * imax
    new_img = (a * img + b).astype(target_type)
    return new_img


In [ ]:
import glob

import cv2
# def run_visualization_repeative():
# # def run_visualization(url):
#   """Inferences DeepLab model and visualizes result."""
#   try:
#     # f = urllib.request.urlopen(url)
#     # jpeg_str = f.read()
#     # original_im = Image.open(BytesIO(jpeg_str))
#     # original_im=Image.open('/content/drive/MyDrive/experiment_mono/1520430162157909301_undistort.png')
#     original_im=Image.open('/content/drive/MyDrive/experiment_mono/1520430162157909301.png')
#     # original_im = Image.open('/home/ziwen/Downloads/DeeplabV3/expriment_mono/1520430162157909301.png')
#     pix = np.array(original_im)
#     imgu8 = convert(pix, 0, 255, np.uint8)
#     undistorted = cv2.remap(imgu8, mapx, mapy, cv2.INTER_LINEAR)
#     plt.imshow(original_im)
#     # pix = np.array(original_im)
#     # plt.imshow(pix)
#     # pix = np.array(original_im)

#     plt.imshow(undistorted)
#     original_im=undistorted

#     color_coverted = cv2.cvtColor(original_im, cv2.COLOR_BGR2RGB)
#     original_im=Image.fromarray(color_coverted)
#     # imgu8 = convert(pix, 0, 255, np.uint8)
#     # original_im = Image.fromarray(imgu8)
#   except IOError:
#     # print('Cannot retrieve image. Please check url: ' + url)
#     return

#   # print('running deeplab on image %s...' % url)
#   resized_im, seg_map = MODEL.run(original_im)

#   # np.savetxt(fname="saved-rain-fall-row-col-names.csv", delimiter=",", X=seg_map)
#   # files.download("saved-rain-fall-row-col-names.csv");
#   # Image.fromarray(seg_map)
#   # seg_im=seg_map.save('result.png')
#   # seg_im.download('result.png')
#   vis_segmentation(resized_im, seg_map)
#   pix = np.array(original_im)
#   plt.imshow(original_im)



# # image_url = IMAGE_URL or _SAMPLE_URL % SAMPLE_IMAGE
# run_visualization_repeative()
# # run_visualization(image_url)


# imageset=glob.glob("/content/drive/MyDrive/tum_vi/data/*.png")
# original_im = Image.open('//tmp/00/image_0/000321.png')
# imageset=glob.glob("/tmp/00/image_0/*.png")
imageset=glob.glob("/tmp/"+dataset+"/image_0/*.png")
imageset_sorted=sorted(imageset)

image_count=len(imageset_sorted)
"$dataset"
# !mkdir /tmp/00/segmentation
!mkdir /tmp/"$dataset"/segmentation

for img in imageset_sorted:
    cv_img = cv2.imread(img)
   
    imgu8 = convert(cv_img, 0, 255, np.uint8)
    
    # try:
    # f = urllib.request.urlopen(url)
    # jpeg_str = f.read()
    # original_im = Image.open(BytesIO(jpeg_str))
    # original_im=Image.open('/content/drive/MyDrive/experiment_mono/1520430162157909301_undistort.png')
    # original_im=Image.open('/content/drive/MyDrive/experiment_mono/1520430162157909301.png')
    # original_im=Image.open('/tmp/dataset-outdoors2_1024_16/mav0/cam0/data/1520430162157909301.png')

    # original_im = Image.open('/home/ziwen/Downloads/DeeplabV3/expriment_mono/1520430162157909301.png')
    # pix = np.array(original_im)
    # imgu8 = convert(pix, 0, 255, np.uint8)
    # undistorted = cv2.remap(imgu8, mapx, mapy, cv2.INTER_LINEAR)
    # plt.imshow(original_im)
    # pix = np.array(original_im)
    # plt.imshow(pix)
    # pix = np.array(original_im)

    # plt.imshow(undistorted)
    # original_im=undistorted

    # color_coverted = cv2.cvtColor(original_im, cv2.COLOR_BGR2RGB)
    # original_im=Image.fromarray(color_coverted)
    # imgu8 = convert(pix, 0, 255, np.uint8)
    original_im = Image.fromarray(imgu8)
    # except IOError:
    # # print('Cannot retrieve image. Please check url: ' + url)
    #   return

    # print('running deeplab on image %s...' % url)
    resized_im, seg_map = MODEL.run(original_im)
    # print(seg_map.dtype)
    seg_img_int8=np.uint8(seg_map)
    seg_im=Image.fromarray(seg_img_int8)
    print(os.path.basename(img))
    # vis_segmentation(resized_im, seg_map)
    # tmp/dataset-outdoors2_1024_16/mav0/cam0/data/
    

    seg_im.save(('/tmp/'+dataset+'/segmentation/'+os.path.basename(img)),"PNG")

    # seg_im.save(('/content/drive/MyDrive/tum_vi/segmentation/'+os.path.basename(img)),"PNG")

    # cv2.imwrite(("/content/drive/MyDrive/tum_vi/segmentation/"+os.path.basename(img)), imgu8)
    image_count=image_count-1
    # print(("./int8_result/"+os.path.basename(img)))
    print("{} {}".format(image_count, "iamges left"))






(1226, 370)
1.0
000000.png
2760 iamges left
(1226, 370)
1.0
000001.png
2759 iamges left
(1226, 370)
1.0
000002.png
2758 iamges left
(1226, 370)
1.0
000003.png
2757 iamges left
(1226, 370)
1.0
000004.png
2756 iamges left
(1226, 370)
1.0
000005.png
2755 iamges left
(1226, 370)
1.0
000006.png
2754 iamges left
(1226, 370)
1.0
000007.png
2753 iamges left
(1226, 370)
1.0
000008.png
2752 iamges left
(1226, 370)
1.0
000009.png
2751 iamges left
(1226, 370)
1.0
000010.png
2750 iamges left
(1226, 370)
1.0
000011.png
2749 iamges left
(1226, 370)
1.0
000012.png
2748 iamges left
(1226, 370)
1.0
000013.png
2747 iamges left
(1226, 370)
1.0
000014.png
2746 iamges left
(1226, 370)
1.0
000015.png
2745 iamges left
(1226, 370)
1.0
000016.png
2744 iamges left
(1226, 370)
1.0
000017.png
2743 iamges left
(1226, 370)
1.0
000018.png
2742 iamges left
(1226, 370)
1.0
000019.png
2741 iamges left
(1226, 370)
1.0
000020.png
2740 iamges left
(1226, 370)
1.0
000021.png
2739 iamges left
(1226, 370)
1.0
000022.png
2738 

In [ ]:
!zip -r /tmp/kitti_segmentation_{dataset}.zip /tmp/{dataset}/segmentation/



In [ ]:
!cp /tmp/kitti_segmentation_{dataset}.zip /content/drive/My\ Drive/kitti/